In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
f = open('liq_ta_game3.json')
data = json.load(f)

In [91]:
players_copy = pd.DataFrame(data['players'])

RangeIndex(start=0, stop=10, step=1)

In [186]:
def get_lane_gold(df_group):
    lane = df_group['lane_role'].iloc[0]
    if lane == 1:
        core = 'carry'
        support = 'hard support'
    elif lane == 2:
        core = 'mid'
        support = None
    elif lane == 3:
        core = 'offlane'
        support = 'soft support'
    booleanMap = {True: core, False: support}
    avg_gold = np.mean(df_group['lane_gold'])
    t_f = df_group['lane_gold'] > avg_gold
    return (df_group['lane_gold']  >= np.mean(df_group['lane_gold'])).map(booleanMap)

In [230]:
pc = players_copy[['name','player_slot','lane_role','gold_t']]
pc.loc[:, 'player_slot'] = pc['player_slot'].apply(lambda x: 'radiant' if x < 5 else 'dire') # classifies as radiant or dire
pc.loc[:,'lane_gold'] = pc['gold_t'].apply(lambda x: x[10]) # [10] to indicate the gold at 10 minutes (end of laning stage)
roles = pc.groupby(['lane_role', 'player_slot']).apply(get_lane_gold).reset_index().rename(columns = {'lane_gold':'role'})
big_roles = pc.merge(right = roles, left_index=True, right_on = 'level_2').drop(columns = ['player_slot_y', 'lane_role_y', 'level_2'])
big_roles = big_roles.rename(columns = {'player_slot_x':'player_slot', 'lane_role_x':'lane_role'})
# pc.merge(roles, on = ['lane_role', 'player_slot'])

c:\Users\aryan\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\Users\aryan\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,name,player_slot,lane_role,gold_t,lane_gold,role
2,Matthew,radiant,1,"[0, 130, 220, 357, 542, 672, 762, 937, 1061, 1...",1507,hard support
8,Sacred,radiant,3,"[0, 130, 549, 798, 1205, 1512, 1762, 2468, 275...",4117,offlane
5,DarkMago♡,radiant,2,"[0, 206, 623, 914, 1355, 1756, 2267, 2856, 325...",4058,mid
9,Panda,radiant,3,"[0, 130, 220, 310, 400, 530, 658, 920, 1173, 1...",1793,soft support
3,Pakazs,radiant,1,"[0, 213, 410, 648, 1062, 1350, 1690, 2026, 242...",3296,carry
0,Insania,dire,1,"[0, 229, 338, 491, 623, 775, 1039, 1174, 1360,...",1740,hard support
6,zai,dire,3,"[0, 247, 599, 834, 1386, 1940, 2384, 2942, 316...",4442,offlane
4,m1CKe,dire,2,"[0, 362, 704, 1004, 1411, 1947, 2322, 2576, 30...",3645,mid
7,Boxi,dire,3,"[0, 210, 300, 463, 836, 1008, 1098, 1548, 1683...",2210,soft support
1,MATUMBAMAN,dire,1,"[0, 248, 527, 835, 1001, 1420, 1811, 2293, 256...",3133,carry


In [231]:
# clean players
def clean_players(df):
    df.loc[:, 'player_slot'] = df['player_slot'].apply(lambda x: 'radiant' if x < 5 else 'dire') # classifies as radiant or dire
    df.loc[:,'lane_gold'] = df['gold_t'].apply(lambda x: x[10]) # [10] to indicate the gold at 10 minutes (end of laning stage)
    
    roles = df.groupby(['lane_role', 'player_slot']).apply(get_lane_gold).reset_index().rename(columns = {'lane_gold':'role'})
    big_roles = df.merge(right = roles, left_index=True, right_on = 'level_2').drop(columns = ['player_slot_y', 'lane_role_y', 'level_2'])
    big_roles = big_roles.rename(columns = {'player_slot_x':'player_slot', 'lane_role_x':'lane_role'})

    return big_roles

In [235]:
players = pd.DataFrame(data['players'])
clean_players(players)[['name','player_slot','role']]

,name,player_slot,role
2,Matthew,radiant,hard support
8,Sacred,radiant,offlane
5,DarkMago♡,radiant,mid
9,Panda,radiant,soft support
3,Pakazs,radiant,carry
0,Insania,dire,hard support
6,zai,dire,offlane
4,m1CKe,dire,mid
7,Boxi,dire,soft support
1,MATUMBAMAN,dire,carry
